# 📦 Phase 1 Proposal

in this exercise, I gained a dataset which contains information about the survivors of the titanic incident. This dataset was provided by the canvas exercise.

from this dataset we have certain features:

1. pnumber: passanger number
2. pclass: class of the passenger ranking from upper middle and lower
3. survived: yes or no
4. name: name of the participant
5. age: age of the passanger
6. embarked: from where they shipped
7. home.dest: where they were headed (not sure)
8. room: room id
9. ticket: ticket id
10. boat: boat id
11. sex: male or female



In [70]:
import sklearn
import pandas
import seaborn

print("scikit-learn version:", sklearn.__version__)     # 1.1.3
print("pandas version:", pandas.__version__)            # 1.5.1
print("seaborn version:", seaborn.__version__)          # 0.12.1

scikit-learn version: 1.1.2
pandas version: 1.5.1
seaborn version: 0.12.1


I start of by printing out some random samples of the dataset and we quickly notice some missing values in certain features. In the tutorial exercies, it is proposed to simply remove these features altogether. With this notebook, I want to see how far I can get without this destructive approach.

In [71]:
df = pandas.read_csv("Titanic.csv")
df.info()
df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pnumber    1313 non-null   int64  
 1   pclass     1313 non-null   object 
 2   survived   1313 non-null   int64  
 3   name       1313 non-null   object 
 4   age        633 non-null    float64
 5   embarked   821 non-null    object 
 6   home.dest  754 non-null    object 
 7   room       77 non-null     object 
 8   ticket     69 non-null     object 
 9   boat       347 non-null    object 
 10  sex        1313 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 113.0+ KB


,pnumber,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
536,537,2nd,1,"Quick, Mrs Frederick C. (Jane Richards)",33.0,Southampton,"Plymouth, Devon / Detroit, MI",NaN,NaN,11,female
1039,1040,3rd,0,"Moutal, Mr Rahamin",NaN,NaN,NaN,NaN,NaN,NaN,male
454,455,2nd,0,"Hunt, Mr George Henry",33.0,Southampton,"Philadelphia, PA",NaN,NaN,NaN,male
939,940,3rd,0,"Kraeff, Mr Theodor",NaN,NaN,NaN,NaN,NaN,NaN,male
264,265,1st,0,"Thorne, Mr George (alias of: Mr George Rosensh...",46.0,Cherbourg,"New York, NY",NaN,NaN,NaN,male


Since there is a lot of missing data we can do the following: Use a new or add new data sources or process the already given data.

for the sake of working with missing values I will do the latter.

Before I start with "fixing" the missing data, I first assess which data is relevant in predicting the survival rate of the passenger. I will first of drop all irrelevant features such as the pnumber, name as they are random and don't improve survival rate.

In [72]:
df.drop(["name", "pnumber"], axis=1, inplace=True)

df

,pclass,survived,age,embarked,home.dest,room,ticket,boat,sex
0,1st,1,29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,1st,0,2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,1st,0,30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,-135,male
3,1st,0,25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,1st,1,0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male
...,...,...,...,...,...,...,...,...,...
1308,3rd,0,NaN,NaN,NaN,NaN,NaN,NaN,male
1309,3rd,0,NaN,NaN,NaN,NaN,NaN,NaN,male
1310,3rd,0,NaN,NaN,NaN,NaN,NaN,NaN,male
1311,3rd,0,NaN,NaN,NaN,NaN,NaN,NaN,female


age may also been a relevant factor since the passengers are all human and have certain cultural and sociatel norms.

for the age it was noted that the value can be noted as a fractional if les than 1 which explains the odd values here in there. we can fix this by multiplying these values by 100.
To fill in the missing values we can do the following methods:

1. imputation with mean or median
2. use a model that predicts the missing age (or make my own model)

In [73]:
df.loc[df['age'] < 1, 'age'] *= 100

df['age'].fillna(df['age'].mean(), inplace=True)

from the previous model it was relevant to use the feature "embarked" as it had a positive influence to be embarked from a certain place. Eventough it seems irrelevant it might have had impact on the passenger as they might have been "more on edge" or sick of the ride. this is all but speculation.

for the feature embarked we can do the following options

1. imputate the data with a default value or most frequent value
2. use another model to predict missing values



In [74]:
df["embarked"].fillna(df["embarked"].mode()[0], inplace=True)

for the room in which the passengers were hosted, I also find this to be relevent information since it may have impacted either the awaereness of the passengers during the accident or hindered this person to reach the boat lessening his survivability rate. 

some tutorials noted that they don't have information on where the rooms were hosted, but using internet we can find some diagrams of how the titanic was split up in. Now that we know where each room can be found at we can use this in our model.

to fill out the missing values, I want to distribute the classess evenly since reports show that the titanic was carrying their on or even above their maximum capacity.

since this data is non numerical, I will have to use a python module to do this for me.

<img src="https://upload.wikimedia.org/wikipedia/commons/0/0d/Olympic_%26_Titanic_cutaway_diagram.png" width=300px>


In [77]:
# strip the first character from the room column
df['room'] = df['room'].str[0]


# fill the missing values with the next value in the cycle evenly distributed
import itertools
unique_rooms = df['room'].dropna().unique()
cycle_rooms = itertools.cycle(unique_rooms)
df['room'] = df['room'].fillna(next(cycle_rooms))

# convert the room column to a categorical column
df = pandas.get_dummies(df, columns=['room'])
df

,pclass,survived,age,embarked,home.dest,ticket,boat,sex,room_2,room_A,room_B,room_C,room_D,room_E,room_F
0,1st,1,29.00000,Southampton,"St Louis, MO",24160 L221,2,female,0,0,1,0,0,0,0
1,1st,0,2.00000,Southampton,"Montreal, PQ / Chesterville, ON",NaN,NaN,female,0,0,0,1,0,0,0
2,1st,0,30.00000,Southampton,"Montreal, PQ / Chesterville, ON",NaN,-135,male,0,0,0,1,0,0,0
3,1st,0,25.00000,Southampton,"Montreal, PQ / Chesterville, ON",NaN,NaN,female,0,0,0,1,0,0,0
4,1st,1,91.67000,Southampton,"Montreal, PQ / Chesterville, ON",NaN,11,male,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,3rd,0,31.80673,Southampton,NaN,NaN,NaN,male,0,0,1,0,0,0,0
1309,3rd,0,31.80673,Southampton,NaN,NaN,NaN,male,0,0,1,0,0,0,0
1310,3rd,0,31.80673,Southampton,NaN,NaN,NaN,male,0,0,1,0,0,0,0
1311,3rd,0,31.80673,Southampton,NaN,NaN,NaN,female,0,0,1,0,0,0,0


# Phase 2 Provisioning


# Phase 3 Predictions